In [60]:
import numpy as np
from scipy.stats import norm
import numpy.linalg


# Testing Zone

### Geometric Brownian motion
Let S be an asset with the following dynamic \\
$$
\frac{dS_t}{S_t} = r dt + \sigma dWt
$$ 
with 
$$
r = 0.05\\
\sigma = 0.2\\
S_0 = 100
$$
Let's consider a european call option on S
$$
Payoff = (S_T - K)_+\\
K = 100\\
T = 1\\
$$


In [56]:
def eu_call_BS(S, T, K, r, q, sigma,t=0):
    d1 = (np.log(S/K) + (r - q + sigma**2/2)*(T-t)) / (sigma*np.sqrt(T-t))
    d2 = d1 - sigma* np.sqrt(T-t)
    return S*np.exp(-q*(T-t)) * norm.cdf(d1) - K * np.exp(-r*(T-t))* norm.cdf(d2)

In [57]:
S = 100
T = 1
K = 100
r = 0.05
q = 0
sigma = 0.2
th_price = eu_call_BS(S, T, K, r, q, sigma)
print("Theoritical Price = %s"%(th_price))

Theoritical Price = 10.450583572185565


Now lets try to price our option with Monte Carlo Using an Euler discretization for the SDE

In [61]:
class WeakConvergenceFailure(Exception):
    pass

def mlmc(Lmin, Lmax, N0, eps, mlmc_fn, alpha_0, beta_0, gamma_0, *args, **kwargs):
    """
    Multilevel Monte Carlo estimation.

    (P, Nl, Cl) = mlmc(...)

    Inputs:
      N0:   initial number of samples    >  0
      eps:  desired accuracy (rms error) >  0
      Lmin: minimum level of refinement  >= 2
      Lmax: maximum level of refinement  >= Lmin

      mlmc_fn: the user low-level routine for level l estimator. Its interface is

        (sums, cost) = mlmc_fn(l, N, *args, **kwargs)

        Inputs:  l: level
                 N: number of paths
                 *args, **kwargs: optional additional user variables

        Outputs: sums[0]: sum(Y)
                 sums[1]: sum(Y**2)
                    where Y are iid samples with expected value
                        E[P_0]            on level 0
                        E[P_l - P_{l-1}]  on level l > 0
                 cost: cost of N samples

      alpha ->  weak error is  O(2^{-alpha*l})
      beta  ->  variance is    O(2^{-beta*l})
      gamma ->  sample cost is O(2^{ gamma*l})

      If alpha, beta are not positive then they will be estimated.

      *args, **kwargs = optional additional user variables to be passed to mlmc_fn

    Outputs:
      P:  value
      Nl: number of samples at each level
      Cl: cost of samples at each level
    """

    # Check arguments

    if Lmin < 2:
        raise ValueError("Need Lmin >= 2")
    if Lmax < Lmin:
        raise ValueError("Need Lmax >= Lmin")
    if N0 <= 0 or eps <= 0:
        raise ValueError("Need N0 > 0, eps > 0")

    # Initialisation

    alpha = max(0, alpha_0)
    beta  = max(0, beta_0)
    gamma = max(0, gamma_0)

    theta = 0.25

    L = Lmin

    Nl   = np.zeros(L+1)
    suml = np.zeros((2, L+1))
    costl = np.zeros(L+1)
    dNl  = N0*np.ones(L+1)

    while sum(dNl) > 0:

        # update sample sums

        for l in range(0, L+1):
            if dNl[l] > 0:
                (sums, cost) = mlmc_fn(l, int(dNl[l]), *args, **kwargs)
                Nl[l]        = Nl[l] + dNl[l]
                suml[0, l]   = suml[0, l] + sums[0]
                suml[1, l]   = suml[1, l] + sums[1]
                costl[l]     = costl[l] + cost

        # compute absolute average, variance and cost

        ml = np.abs(suml[0, :]/Nl)
        Vl = np.maximum(0, suml[1, :]/Nl - ml**2)
        Cl = costl/Nl

        # fix to cope with possible zero values for ml and Vl
        # (can happen in some applications when there are few samples)

        for l in range(3, L+2):
            ml[l-1] = max(ml[l-1], 0.5*ml[l-2]/2**alpha)
            Vl[l-1] = max(Vl[l-1], 0.5*Vl[l-2]/2**beta)

        # use linear regression to estimate alpha, beta, gamma if not given
        if alpha_0 <= 0:
            A = np.ones((L, 2)); A[:, 0] = range(1, L+1)
            x = np.linalg.lstsq(A, np.log2(ml[1:]))[0]
            alpha = max(0.5, -x[0])

        if beta_0 <= 0:
            A = np.ones((L, 2)); A[:, 0] = range(1, L+1)
            x = np.linalg.lstsq(A, np.log2(Vl[1:]))[0]
            beta = max(0.5, -x[0])

        if gamma_0 <= 0:
            A = np.ones((L, 2)); A[:, 0] = range(1, L+1)
            x = np.linalg.lstsq(A, np.log2(Cl[1:]))[0]
            gamma = max(0.5, x[0])

        # set optimal number of additional samples

        Ns = np.ceil( np.sqrt(Vl/Cl) * sum(np.sqrt(Vl*Cl)) / ((1-theta)*eps**2) )
        dNl = np.maximum(0, Ns-Nl)

        # if (almost) converged, estimate remaining error and decide
        # whether a new level is required

        if sum(dNl > 0.01*Nl) == 0:
            # 23/3/18 this change copes with cases with erratic ml values
            rang = list(range(min(3, L)))
            rem = ( np.amax(ml[[L-x for x in rang]] / 2.0**(np.array(rang)*alpha))
                    / (2.0**alpha - 1.0) )
            # rem = ml[L] / (2.0**alpha - 1.0)

            if rem > np.sqrt(theta)*eps:
                if L == Lmax:
                    raise WeakConvergenceFailure("Failed to achieve weak convergence")
                else:
                    L = L + 1
                    Vl = np.append(Vl, Vl[-1] / 2.0**beta)
                    Nl = np.append(Nl, 0.0)
                    suml = np.column_stack([suml, [0, 0]])
                    Cl = np.append(Cl, Cl[-1]*2**gamma)
                    costl = np.append(costl, 0.0)

                    Ns = np.ceil( np.sqrt(Vl/Cl) * sum(np.sqrt(Vl*Cl))
                            / ((1-theta)*eps**2) )
                    dNl = np.maximum(0, Ns-Nl)

    # finally, evaluate the multilevel estimator
    P = sum(suml[0,:]/Nl)

    return (P, Nl, Cl)


Finally, let's price the option with MLMC

In [89]:
def mlmc_fn(l,N):
    S0 = 100
    sigma = 0.2
    r = 0.05
    T = 1
    K = 100
    dt = 2**(-l)
    sumY = 0
    sumY2 = 0
    for i in range(N):
        Sf = S0
        Sc = S0
        dWf = np.sqrt(dt) * np.random.randn(int(T/dt))
        for i in range(int(T/dt)):
            Sf += r * Sf * dt + sigma * Sf * dWf[i]
        Pf = np.max(Sf - K, 0)
        
        if l == 0 :
            Pc = 0
        else :
            #dWc = dWf.reshape(-1, 2)
            dWc = np.sqrt(dt) * np.random.randn(int(T/dt/2))
            for i in range(int(T/dt/2)):
                Sc += r * Sc * dt + sigma * Sc * dWc[i]
            Pc = np.max(Sc - K, 0)
        
        Y = Pf# - Pc
        sumY += Y
        sumY2 += Y**2
        
    return [sumY, sumY2], 0

In [90]:
print(mlmc_fn(10, 10000)[0][0]/10000)

5.129438893222786


In [94]:
Lmin = 2
Lmax = 1000
N0 = 1e04
eps = 1e-06
gamma_0 = 2
alpha_0 = 1
beta_0 = 1

mlmc_price = mlmc(Lmin, Lmax, N0, eps, mlmc_fn, alpha_0, beta_0, gamma_0)[0]
print(mlmc_price)

nan


C:\Users\oussama\AppData\Local\Temp\ipykernel_17464\1355844043.py:112: RuntimeWarning: divide by zero encountered in true_divide
  Ns = np.ceil( np.sqrt(Vl/Cl) * sum(np.sqrt(Vl*Cl)) / ((1-theta)*eps**2) )
C:\Users\oussama\AppData\Local\Temp\ipykernel_17464\1355844043.py:112: RuntimeWarning: invalid value encountered in multiply
  Ns = np.ceil( np.sqrt(Vl/Cl) * sum(np.sqrt(Vl*Cl)) / ((1-theta)*eps**2) )
C:\Users\oussama\AppData\Local\Temp\ipykernel_17464\1355844043.py:136: RuntimeWarning: divide by zero encountered in true_divide
  Ns = np.ceil( np.sqrt(Vl/Cl) * sum(np.sqrt(Vl*Cl))
C:\Users\oussama\AppData\Local\Temp\ipykernel_17464\1355844043.py:136: RuntimeWarning: invalid value encountered in multiply
  Ns = np.ceil( np.sqrt(Vl/Cl) * sum(np.sqrt(Vl*Cl))
C:\Users\oussama\AppData\Local\Temp\ipykernel_17464\1355844043.py:141: RuntimeWarning: invalid value encountered in true_divide
  P = sum(suml[0,:]/Nl)
